<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
#
# address = "University/Term 6/MIR/Homeworks/HW3"
# import sys
# sys.path.append(f"/content/drive/My Drive/{address}")
#
# %cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


In [1]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install fasttext
# !pip install scikit-learn
# !pip install pandas
# !pip install nltk
# !pip install torch
# !pip install transformers
# !pip install sentence_transformers
# !pip install faiss-cpu
# !pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 7.6 MB/s 
     |████████████████████████████████| 1.4 MB 33.9 MB/s 
     |████████████████████████████████| 233 kB 43.6 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=97989bd5f53a136471c38e3f303898a2a169a8087ef69da07b244e964971a626
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153378 sha256=9007766b9bcffdcb6c36385b2062acc77b12870f59dd03b701e0143a40d64c7f
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Looking in indexes: ht

In [2]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import faiss
import pickle
import zipfile
import requests
# import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from transformers import pipeline
from IPython.display import display
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [3]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [4]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [5]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [9]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [10]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [11]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]


def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم.
    </p>
</div>

In [12]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


preprocessor = Preprocessor(stopwords_path='stopwords.txt')

texts = [data_to_text(data) for _, data in dataset.iterrows()]
preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
save_preprocessed_texts(preprocessed_texts)

# preprocessed_texts = load_preprocessed_texts()

100%|██████████| 68362/68362 [07:04<00:00, 161.20it/s]


In [13]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


mini_dataset, mini_preprocessed_texts = get_mini_dataset()

100%|██████████| 4000/4000 [00:24<00:00, 162.38it/s]


# Models
## Boolean

In [30]:
QUERIES = [
    "وزارت خارجه در برجام",
    "حمله روسیه به اوکراین",
    "سقوط بورس",
    "اعلام جزئیات زلزله",
    "اقدامات شهرداری تهران",
    "صعود ایران به جام جهانی",
    "ماسک توئیتر را خرید",
    "جشنواره فیلم فجر",
    "نمایشگر اولد سامسونگ",
    "طرز تهیه دسر",
]

In [21]:
class BooleanIR:

    def __init__(self):
        self.vectorizer = CountVectorizer(binary=True)
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def preprocess_query(self, query):
        preprocessor = Preprocessor(stopwords_path='stopwords.txt')
        preprocessed_query = []
        for subquery in query:
            if subquery[0] == '-':
                preprocessed_query.append('-' + (' '.join(preprocessor.preprocess(subquery[1:]))))
            else:
                preprocessed_query.append(' '.join(preprocessor.preprocess(subquery)))
        return preprocessed_query

    def predict(self, query, dataset, k):
        query = self.preprocess_query(query)
        query_result = []
        query_result_BUT = []
        for subquery in query:
            is_subquery_but = False
            if subquery[0] == '-':
                is_subquery_but = True
                subquery = subquery[1:]
            query_transform = self.vectorizer.transform([subquery]).todense().tolist()[0]
            query_indices = np.nonzero(query_transform)[0]
            for index, doc in self.dense_vectors_df.iterrows():
                if all(np.take(doc, query_indices)):
                    if is_subquery_but:
                        query_result_BUT.append(index)
                    else:
                        query_result.append(index)
        query_result = list(set(query_result) - set(query_result_BUT))
        k = min(k, len(query_result))
        return dataset.iloc[query_result[:k]]

    def save_boolean_model(self, path='BooleanIR_model.pickle'):
        with open(path, "wb") as file:
            pickle.dump(self, file)

    def load_boolean_model(self, path='BooleanIR_model.pickle'):
        with open(path, "rb") as file:
            return pickle.load(file)

    def prepare(self, dataset, mode, save=False):
        model = None
        if mode == 'train':
            self.fit_transform_vectorizer(dataset)
            model = self
        if mode == 'load':
            model = self.load_boolean_model()
        if save:
            self.save_boolean_model()
        return model

In [25]:
booleanIR_model = BooleanIR().prepare(mini_preprocessed_texts, mode='load', save=False)

In [31]:
for i, query in enumerate(QUERIES):
    print(f"Query {i+1}: {query}")
    display(booleanIR_model.predict([query], mini_dataset, k=10))

Query 1: وزارت خارجه در برجام


,date,title,intro,body,category
67,شنبه ۹ بهمن ۱۴۰۰ - ۱۲:۴۷,اظهارات مقام آمریکایی درباره دلیل توقف مذاکرات...,یک مقام وزارت خارجه آمریکا در پی وقفه چند روزه...,به گزارش همشهری آنلاین به نقل از برنا، یک سخنگ...,Politics
196,شنبه ۲۷ فروردین ۱۴۰۱ - ۱۸:۱۰,هشدار نشنال اینترست درباره درگیری فاجعه بار آم...,یک مجله آمریکایی در گزارشی با پرداختن به وقفه ...,به گزارش همشهری‌آنلاین به نقل از ایسنا، مجله آ...,Politics
390,سه‌شنبه ۹ آذر ۱۴۰۰ - ۱۴:۴۰,آغاز گفت‌وگوهای کارشناسی درباره لغو تحریم‌ها د...,کارگروه لغو تحریم‌های ظالمانه و غیرقانونی آمری...,به گزارش همشهری‌آنلاین به نقل از ایرنا، این کا...,Politics
136,دوشنبه ۱۵ فروردین ۱۴۰۱ - ۰۸:۰۰,اسرائیل برای درگیری نظامی ایران با آمریکا تلا...,رضا نصری در پاسخ به این که مهم‌ترین چالش‌های پ...,یک کارشناس ارشد مسائل بین الملل، با بیان این ک...,Politics
204,سه‌شنبه ۲۸ دی ۱۴۰۰ - ۱۴:۰۹,یعنی چه که به سمت روسیه غش کرده‌ایم؟ | روابط ب...,علی اکبر ولایتی معتقد است: ما تابه‌حال از داشت...,به گزارش همشهری آنلاین، روزنامه کیهان نوشت: ۱۶...,Politics
208,چهارشنبه ۲۰ بهمن ۱۴۰۰ - ۱۶:۵۵,مانع اصلی دولت بایدن برای بازگشت به برجام | جا...,معاون سخنگوی‌ وزارت امور خارجه آمریکا گفت واشن...,به گزارش همشهری‌آنلاین به نقل از فارس، «جالینا...,Politics
81,جمعه ۲۷ اسفند ۱۴۰۰ - ۱۳:۵۳,گزارش محرمانه مقام‌های آمریکایی به کنگره دربار...,دو مقام ارشد دولت آمریکا در جلسه محرمانه کنگره...,به گزارش همشهری آنلاین به نقل از فارس، نشریه «...,Politics
84,سه‌شنبه ۴ خرداد ۱۴۰۰ - ۲۲:۵۱,در مذاکرات وین چه گذشت؟| گزارش عراقچی,دور پنجم گفتگوها در خصوص لغو تحریم های آمریکا ...,به گزارش همشهری آنلاین، در این نشست که با حضور...,Politics
22,پنجشنبه ۵ اسفند ۱۴۰۰ - ۰۸:۲۲,ادعای کاخ سفید درباره توافق با ایران | ما به ه...,سخنگوی کاخ سفید ادعا کرد با توجه به سرعت پیشرف...,به گزارش همشهری آنلاین به نقل از فارس، «جنیفر ...,Politics
214,دوشنبه ۱۰ آبان ۱۴۰۰ - ۱۴:۵۱,چین: آمریکا باید سیاست اشتباه «فشار حداکثری» ع...,سخنگوی وزارت امور خارجه چین با تأکید بر حمایت ...,به گزارش همشهری‌آنلاین به نقل از فارس، «وانگ و...,Politics


Query 2: حمله روسیه به اوکراین


,date,title,intro,body,category
512,چهارشنبه ۲۵ اسفند ۱۴۰۰ - ۲۱:۱۸,آمریکا سلاح‌های ضدهواپیما و ضدتانک و پهپادهای ...,جو بایدن، رئیس‌جمهور آمریکا روز چهارشنبه گفت ا...,به گزارش همشهری آنلاین به نقل از آسوشیتدپرس ای...,World
1024,شنبه ۲۰ فروردین ۱۴۰۱ - ۱۰:۳۱,سرعت افزایش قیمت نفت گرفته شد,قیمت نفت روز جمعه دو درصد افزایش یافت اما تحت ...,به گزارش همشهری آنلاین به نقل از ایسنا، بهای م...,Economy
768,جمعه ۶ اسفند ۱۴۰۰ - ۰۲:۴۷,روسیه به دنبال این هدف در اوکراین است | تعداد ...,یک مقام دفاعی ایالات متحده می‌گوید، کی‌یف یکی ...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,World
771,چهارشنبه ۲۵ اسفند ۱۴۰۰ - ۱۹:۲۶,ادعای ناتو درباره شرایط نیروهای روسیه | به ارس...,مقامات ارشد نظامی ناتو مدعی شده‌اند که ولادیمی...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World
2309,پنجشنبه ۸ اردیبهشت ۱۴۰۱ - ۰۸:۲۳,هیچ وقت گروه به این خوبی نداشتیم | حسم می گوید...,رضا قوچان‌نژاد یک سالی هست از میادین فوتبال دو...,به گزارش همشهری آنلاین گوچی به‌خاطر بازی‌های ...,Sport
518,یکشنبه ۷ فروردین ۱۴۰۱ - ۰۸:۵۹,ببینید | حمله موشک‌های «کالیبر» روسیه به انبار...,وزارت دفاع روسیه می‌گوید ارتش این کشور بامداد ...,به گزارش همشهری آنلاین، وزارت دفاع روسیه تصریح...,World
774,چهارشنبه ۱۱ اسفند ۱۴۰۰ - ۱۹:۲۴,ترکیه می‌گوید روسیه عبور کشتی‌های جنگی‌اش به د...,وزیر خارجه ترکیه می‌گوید روسیه به تقاضای این ک...,به گزارش همشهری آنلاین به نقل از رویترز مولود ...,World
520,پنجشنبه ۴ فروردین ۱۴۰۱ - ۱۸:۱۵,هشدار کرملین به بوریس جانسون | پیش‌بینی روسیه ...,کرملین می‌گوید که بوریس جانسون، نخست وزیر بریت...,به گزارش همشهری آنلاین و به نقل از خبرگزاری ری...,World
777,پنجشنبه ۵ اسفند ۱۴۰۰ - ۲۲:۵۵,راز تعریف و تمجیدهای ترامپ از پوتین چیست؟,بارها و بارها در این هفته و در حالی که روسیه ن...,به گزارش همشهری آنلاین و به نقل از شبکه خبری س...,World
3594,یکشنبه ۱ اسفند ۱۴۰۰ - ۱۱:۴۵,حمله به اینترنت؛ نخستین هدف ارتش روسیه در اوکراین,اتفاقات رخ داده در قزاقستان تا حدودی می‌تواند ...,به گزارش همشهری آنلاین و به نقل از روزنامه نیو...,IT


Query 3: سقوط بورس و اعتراض مردم


...

,date,title,intro,body,category
2826,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۰:۲۴,تصویر شهید حاج قاسم سلیمانی روی سیمرغ‌های جشنو...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، سیمرغ های جشنواره چهلم...,Culture
3083,شنبه ۶ آذر ۱۴۰۰ - ۱۲:۵۲,قصر شیرین به تلویزیون می‌آید,منتخبی از فیلم‌های سیدرضا میرکریمی شامل «زیر ن...,به گزارش همشهری آنلاین به نقل از پایگاه اطلاع ...,Culture
3978,چهارشنبه ۱۳ بهمن ۱۴۰۰ - ۰۰:۰۶,عکس | ژست متفاوت صبا سهیلی کنار سینا مهراد در ...,تصاویری از سینا مهراد و خواهرش صبا سهیلی را مش...,به گزارش همشهری آنلاین، در دومین روز برگزاری چ...,LifeSkills
2957,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۶:۵۲,«شهربانو» بهترین فیلم جشنواره سیدنی شد,فیلم سینمایی «شهربانو» ساخته مریم بحرالعلومی ب...,به گزارش همشهری آنلاین، فیلم سینمایی «شهربانو»...,Culture
2834,دوشنبه ۲۷ دی ۱۴۰۰ - ۱۲:۱۰,مریلا زارعی و بهروز شعیبی با هناس به جشنواره م...,نام فیلم سینمایی «صورت فلکی» به «هِناس» تغییر ...,به گزارش همشهری آنلاین به نقل از روابط عمومی پ...,Culture
2841,جمعه ۱۳ فروردین ۱۴۰۰ - ۱۲:۳۳,خورشید مجیدی در اسپانیا روی پرده می‌رود,فیلم سینمایی «خورشید» به کارگردانی مجید مجیدی ...,به گزارش همشهری آنلاین، کمتر از ۴۵ روز مانده ب...,Culture
2972,دوشنبه ۲۰ دی ۱۴۰۰ - ۱۶:۳۹,زندگینامه: ساره بیات (۱۳۵۸-),ساره بیات در سال ۱۳۵۸ در مشهد بدنیا آمد.,به گزارش همشهری آنلاین، ساره بیات، زاده ۱۴ مه...,Culture
2976,شنبه ۹ بهمن ۱۴۰۰ - ۱۴:۳۴,نخستین عکس از رعنا آزادی‌ور در فیلم مرد بازنده,رعنا آزادی‌ور در دومین همکاری با محمدحسین مهدو...,به گزارش همشهری‌آنلاین به نقل از مهر، فیلمنامه...,Culture
3104,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۲:۵۷,لحظه شماری می‌کردیم رهبر انقلاب این فیلم را بب...,تهیه‌کننده فیلم «موقعیت مهدی» از نگاه حکیمانه ...,به گزارش همشهری‌آنلاین به نقل از تسنیم، پیمان ...,Culture
3108,یکشنبه ۸ فروردین ۱۴۰۰ - ۰۸:۱۷,زندگینامه: هانیه توسلی (۱۳۵۸-),هانیه توسلی سال ۱۳۵۸ در همدان به دنیا آمد.,به گزارش همشهری آنلاین، هانیه توسلی زادهٔ ۱۴ ...,Culture


Query 9: نمایشگر اولد گوشی هوشمند


,date,title,intro,body,category


Query 10: طرز تهیه دسر


,date,title,intro,body,category
3714,چهارشنبه ۳ شهریور ۱۴۰۰ - ۱۴:۲۴,طرز تهیه حلوای شیر | نکات پخت حلوا با طعمی متفاوت,اگر تصمیم‌ گرفته‌اید حلوا بپزید زمانی را امتحا...,به گزارش همشهری آنلاین به نقل از ایرنا، حلوا ا...,LifeSkills
3619,شنبه ۲۲ خرداد ۱۴۰۰ - ۱۲:۰۰,طرز تهیه کوکی شکلاتی در خانه | نکات طلایی برای...,کوکی شکلاتی از شیرینی‌های پرطرفدار است. در این...,به گزارش همشهری آنلاین، کوکی شکلاتی هم مانند ک...,LifeSkills
3622,شنبه ۸ خرداد ۱۴۰۰ - ۱۱:۳۹,طرز تهیه بستنی توت‌فرنگی | ترفندی برای یخ نزدن...,بستنی توت‌فرنگی یکی از خوشمزه‌ترین و محبوب‌تری...,به گزارش همشهری آنلاین، توت‌فرنگی یکی از خوشمز...,LifeSkills
3958,یکشنبه ۱۳ تیر ۱۴۰۰ - ۲۲:۰۰,فواید شگفت‌انگیز ژلاتین برای بدن | چگونه نوشید...,احتمالا برایتان جالب خواهد بود که با یک نوشیدن...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,LifeSkills
3914,شنبه ۲۲ خرداد ۱۴۰۰ - ۱۲:۱۰,طرز تهیه پاناکوتا | نکات کلیدی برای درست کردن ...,پاناکوتا این دسر جذاب ایتالیایی مدت‌هاست بین م...,به گزارش همشهری آنلاین، دسرها بدون شک، هیجان‌ا...,LifeSkills
3999,پنجشنبه ۲۶ فروردین ۱۴۰۰ - ۱۸:۲۶,طرز تهیه فنجان خیار | راهی برای تامین آب بدن د...,یکی از مهم‌ترین نکته‌ها برای حفظ سلامت جسم در ...,به گزارش همشهری‌آنلاین، در ماه مبارک رمضان‌ ه...,LifeSkills


## TF-IDF

In [33]:
class TF_IDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def predict(self, query, dataset, k):
        query = ' '.join(Preprocessor(stopwords_path='stopwords.txt').preprocess(query))
        query_transform = self.vectorizer.transform([query]).todense().tolist()[0]
        dense_vectors = self.dense_vectors_df.values.tolist()
        df_cosine_sim = list(map(lambda doc: self.cosine_sim(query_transform, doc), dense_vectors))
        self.dense_vectors_df['query_sim'] = df_cosine_sim
        indices = self.dense_vectors_df.nlargest(k, 'query_sim').index
        self.dense_vectors_df = self.dense_vectors_df.drop(columns=['query_sim'])
        return dataset.iloc[indices]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "wb") as file:
            pickle.dump(self, file)

    def load_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "rb") as file:
            return pickle.load(file)

    def prepare(self, dataset, mode, save=False):
        model = None
        if mode == 'train':
            self.fit_transform_vectorizer(dataset)
            model = self
        if mode == 'load':
            model = self.load_TF_IDF_model()
        if save:
            self.save_TF_IDF_model()
        return model

In [35]:
TF_IDF_model = TF_IDF().prepare(mini_preprocessed_texts, mode='train', save=True)

In [ ]:
for i, query in enumerate(QUERIES):
    print(f"Query {i+1}: {query}")
    display(TF_IDF_model.predict(query, mini_dataset, k=10))

Query 1: وزارت خارجه در برجام


,date,title,intro,body,category
282,سه‌شنبه ۱۷ فروردین ۱۴۰۰ - ۲۳:۴۱,جهانگیری: فراموش نکرده‌ایم عده‌ای در مجلس برجا...,جهانگیری، معاون اول رئیس‌جمهوری، گفت: فراموش ن...,به گزارش همشهری‌آنلاین به نقل از انتخاب، جهانگ...,Politics
170,یکشنبه ۱۰ مرداد ۱۴۰۰ - ۱۳:۲۴,توصیه ظریف به وزیر خارجه دولت رئیسی | واردات چ...,وزیر امور خارجه با بیان اینکه «من شهادت می‌دهم...,به گزارش همشهری آنلاین به نقل از مهر، محمدجوا...,Politics
214,دوشنبه ۱۰ آبان ۱۴۰۰ - ۱۴:۵۱,چین: آمریکا باید سیاست اشتباه «فشار حداکثری» ع...,سخنگوی وزارت امور خارجه چین با تأکید بر حمایت ...,به گزارش همشهری‌آنلاین به نقل از فارس، «وانگ و...,Politics
134,چهارشنبه ۷ مهر ۱۴۰۰ - ۲۳:۲۴,حضور سردار قاآنی در وزارت خارجه و دیدار با امی...,سردار قاآنی با حضور در وزارت خارجه و تبریک به ...,به گزارش همشهری‌آنلاین به نقل از فارس، سردار ا...,Politics
35,پنجشنبه ۸ مهر ۱۴۰۰ - ۱۵:۱۶,موضع ایران در برابر طالبان چیست؟ | خطیب‌زاده: ...,​ سخنگوی وزارت امور خارجه با تاکیدبراینکه اروپ...,به گزارش همشهری‌آنلاین به نقل از اداره کل اطل...,Politics
19,پنجشنبه ۱۸ آذر ۱۴۰۰ - ۱۶:۲۷,بررسی متون پیشنهادی ایران در مذاکرات امروز وین...,نشست کمیسیون مشترک برجام با محور بررسی متون پی...,به گزارش همشهری آنلاین، نشست کمیسیون مشترک برج...,Politics
172,یکشنبه ۱۵ فروردین ۱۴۰۰ - ۱۵:۱۵,ظریف و همتای انگلیسی گفت‌وگو کردند | شرط ایران...,روز گذشته محمدجواد ظریف، وزیر امور خارجه جمهور...,به گزارش همشهری‌آنلاین، طرفین در این گفت‌وگو آ...,Politics
398,یکشنبه ۱۴ شهریور ۱۴۰۰ - ۱۲:۰۸,نماینده تهران در مجلس: ما از مذاکره نمی‌ترسیم ...,نماینده تهران در مجلس گفت: در مجلس نهم مخالفان...,به گزارش همشهری آنلاین به نقل از برنا، اسماعیل...,Politics
77,چهارشنبه ۹ تیر ۱۴۰۰ - ۱۹:۱۷,آغاز نشست شورای امنیت سازمان ملل درباره برجام,نشست شورای امنیت سازمان ملل امروز چهارشنبه برا...,به گزارش همشهری آنلاین به نقل از ایرنا، رز ما...,Politics
109,پنجشنبه ۲۶ فروردین ۱۴۰۰ - ۱۶:۱۴,مذاکرات در وین از سر گرفته شد,نشست کمیسیون مشترک برجام ظهر روز پنج‌شنبه در س...,به گزارش همشهری‌آنلاین به نقل از ایسنا، این نش...,Politics


Query 2: حمله روسیه به اوکراین


,date,title,intro,body,category
733,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۰:۲۸,ببینید | اوکراین؛ از آژیر حمله هوایی تا آتش سو...,رئیس جمهوری روسیه صبح امروز اعلام کرد که در پا...,به گزارش همشهری آنلاین به نقل از تسنیم، شورای ...,World
501,یکشنبه ۱ اسفند ۱۴۰۰ - ۰۸:۴۹,هشدار درباره نزدیکی جنگ جهانی سوم | روسیه قصد ...,بوریس جانسون، نخست وزیر انگلیس هشدار داد که وض...,به گزارش همشهری آنلاین به نقل از ایسنا، جانسون...,World
444,دوشنبه ۲۵ بهمن ۱۴۰۰ - ۰۸:۲۴,روسیه ژست حمله به اوکراین را گرفت | شاید چین ه...,یک استاد ژئوپلیتیک دانشگاه با اشاره به اینکه ر...,به گزلرش همشهری آنلاین، عبدالرضا فرجی راد دربا...,World
3271,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
571,یکشنبه ۲۴ بهمن ۱۴۰۰ - ۱۱:۲۹,حمله قریب‌الوقوع روسیه به اوکراین؟,یک مقام کاخ الیزه می‌گوید در تماس تلفنی رئیس ج...,به گزارش همشهری آنلاین به نقل از فارس، یک مقام...,World
760,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۳:۱۱,تازه‌ترین درخواست وزیر امور خارجه اوکراین علیه...,دیمیترو کولبا، وزیر امور خارجه اوکراین، پس از ...,به گزارش همشهری آنلاین و به نقل از گاردین، او ...,World
764,جمعه ۶ اسفند ۱۴۰۰ - ۱۲:۱۹,ادعای وزیر دفاع انگلیس درباره نقشه روسیه برای ...,وزیر دفاع انگلیس در بیانیه‌ای مدعی شد، روسیه ق...,به گزارش همشهری آنلاین و به نقل از گاردین، بن ...,World
578,جمعه ۶ اسفند ۱۴۰۰ - ۱۶:۰۷,پوتین: آماده مذاکره با اوکراین هستیم | موضع ری...,رئیس جمهور چین در یک تماس تلفنی با رئیس جمهور ...,به گزارش همشهری‌آنلاین به نقل از فارس، رسانه‌ه...,World
589,شنبه ۱۷ اردیبهشت ۱۴۰۱ - ۱۶:۲۷,جنگ اوکراین روزانه چقدر برای روسیه آب می‌خورد؟,یک نشریه آمریکایی در گزارشی با اشاره به سومین ...,به گزارش همشهری‌آنلاین، ایرنا به نقل از نشریه ...,World
460,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۶:۱۰,منتظر جنگ جهانی سوم باشیم؟ | امکان چریکی شدن ج...,جهانگیر کرمی، عضو هیات علمی دانشگاه تهران و کا...,همشهری‌آنلاین - اصغر صوفی: درگیری میان اوکراین...,World


## Transformer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
         ترنسفرمر
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش یک کلاس تعریف می‌کنیم که بتواند کارهایی که نیاز داریم را با استفاده از مدلی بر پایه‌ی ترنسفرمرها انجام دهد. برای این کار از SentenceTransformer در کتاب‌خانه‌ی HuggingFace بهره می‌بریم و مدل ParsBERT که به دست هوش‌واره فراهم شده است را به عنوان مدل زبانی آن استفاده می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در آغاز که یک شی با استفاده از این کلاس ساخته می‌شود یک SentenceTransformer با استفاده از مدل ParsBERT می‌سازیم تا از آن برای به دست آوردن embeddingها استفاده کنیم. سپس در صورتی که GPU در دسترس باشد این مدل را به روی آن منتقل می‌کنیم. علاوه بر این می‌توانیم یک پیش‌پردازشگر هم به متد سازنده بدهیم. این همان پیش‌پردازشگری است که دیتاست اولیه با آن پردازش شده است. ابزار ما از آن برای پیش‌پردازش درخواست‌ها در زمان تشخیص و تبدیل آن‌ها به فرمتی مطابق با متن اولیه پس از پیش‌پردازش استفاده خواهد کرد.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در تابع train_embeddings لیستی از متن‌ها یا لیستی از متن‌های شکسته شده به کلمات را دریافت می‌کنیم تا امبدینگ‌های مربوط به آن‌ها را به دست بیاوریم. در حالت دوم، پیش از ادامه‌ی کار کلمات را در یک متن واحد در کنار هم قرار می دهیم. توجه می‌کنیم که داده‌ها پس از پیش‌پردازش در حالت دوم قرار دارند. در نهایت پس از به دست آمدن امبدینگ‌ها، آن‌ها را در یک آرایه‌ی نامپای ذخیره می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از به دست آمدن امبدینگ‌ها می‌توانیم آن‌ها را با استفاده از متد save_embeddings در آدرس دلخواهمان به عنوان یک فایل ذخیره کنیم و در دفعات بعدی به جای محاسبه‌ی دوباره، آن‌ها را با استفاده از متد load_embeddings بازیابی کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        وقتی که امبدینگ‌ها را محاسبه یا بازیابی کردیم، می‌توانیم با استفاده از متد make_index نمایه‌هایی را با استفاده از ابزار Faiss ایجاد کنیم. این ابزار در به دست آوردن موارد مشابه به درخواست‌ها به ما کمک خواهد کرد.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در نهایت پس از اینکه تمام مراحل گفته شده را طی کردیم می‌توانیم با استفاده از متد predict یک درخواست را به ابزارمان بدهیم و این ابزار پس از به دست آوردن امبدینگ مربوط به آن درخواست، تعداد دلخواهی از موارد مربوط به آن را با توجه به مدل ترنسفرمر استفاده شده و نمایه‌ی Faiss تولید شده به ما خواهد داد.
    </p>
</div>

In [ ]:
class Transformer:

    def __init__(self, preprocessor=None):
        self.model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
        self.preprocessor = preprocessor
        if torch.cuda.is_available():
            self.model = self.model.to(torch.device('cuda'))
        self.embeddings = None
        self.index = None

    def train_embeddings(self, train_dataset: list):
        if type(train_dataset[0]) == list:
            train_dataset = list(map(lambda doc: ' '.join(doc), train_dataset))
        self.embeddings = self.model.encode(train_dataset, show_progress_bar=True)
        self.embeddings = np.array(
            [embedding for embedding in self.embeddings]).astype('float32')

    def make_index(self, dataset: list):
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index = faiss.IndexIDMap(self.index)
        self.index.add_with_ids(self.embeddings, np.arange(len(dataset)).astype('int64'))

    def save_embeddings(self, path='Transformer_model.pickle'):
        with open(path, 'wb') as file:
            pickle.dump(self.embeddings, file)

    def load_embeddings(self, path='Transformer_model.pickle'):
        with open(path, 'rb') as file:
            self.embeddings = pickle.load(file)

    def predict(self, query: str, dataset: pd.DataFrame, k=10):
        if self.preprocessor:
            query = ' '.join(self.preprocessor.preprocess(query))
        vector = self.model.encode(list([query]))
        D, I = self.index.search(np.array(vector).astype('float32'), k=k)
        return dataset.iloc[I.flatten().tolist()]

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train_embeddings(dataset)
        if mode == 'load':
            self.load_embeddings()
        if save:
            self.save_embeddings()
        transformer.make_index(dataset)


In [ ]:
transformer = Transformer(preprocessor=preprocessor)
transformer.prepare(preprocessed_texts, 'load')

Some weights of the model checkpoint at C:\Users\Mohammad/.cache\torch\sentence_transformers\HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at C:\Users\Mohammad/.cache\torch\sentence_t

In [ ]:
transformer.predict("صعود به جام جهانی", dataset)
transformer.predict("نمایشگر اولد", dataset)

,date,title,intro,body,category
63212,جمعه ۱۵ بهمن ۱۴۰۰ - ۱۵:۰۹,روش خاص اپل برای خنک نگه داشتن مک بوک پرو ۲۰۲۱,در اوایل این هفته، اپل یک سند پشتیبانی جدید در...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
63431,چهارشنبه ۲۸ اردیبهشت ۱۴۰۱ - ۰۸:۱۹,اپل در حال آزمایش گوشی هوشمند تاشو | استفاده ا...,شرکت اپل در حال طراحی یک گوشی هوشمند تاشو است ...,به‌گزارش همشهری آنلاین به نقل از گیزموچاینا، ا...,IT
63341,دوشنبه ۲ اسفند ۱۴۰۰ - ۱۰:۱۷,بحران در تولید نمایشگرهای آیفون | اپل راه چاره...,شرکتBOE که یکی از تولیدکنندگان پیشرو پنل‌های ...,به گزارش همشهری آنلاین و به نقل از گیزموچاینا،...,IT
55526,چهارشنبه ۳ آذر ۱۴۰۰ - ۰۷:۳۵,"خودرویی با ""داشبورد خجالتی""!","نوآوری جدید شرکت ""کانتیننتال""(Continental) موس...",به گزارش همشهری آنلاین، ایسنا به نقل از آی‌ای،...,Science
63143,سه‌شنبه ۱۴ دی ۱۴۰۰ - ۲۰:۳۰,عکس | نخستین تلویزیون QD-OLED سونی به بازار می...,سونی انتظارات فراوانی را با مجموعه تلویزیون‌ها...,به گزارش همشهری آنلاین و به نقل از وب‌سایت ورج...,IT
63125,دوشنبه ۲۰ دی ۱۴۰۰ - ۱۴:۲۸,مشخصات صفحه نمایش‌ جدید سامسونگ فاش شد,مشخصات کلیدی QD-OLED جدید سامسونگ دیسپلی توسط ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت ورج...,IT
63137,پنجشنبه ۱۶ دی ۱۴۰۰ - ۰۱:۴۲,تصاویر | نمایش خیره‌کننده سامسونگ از گوشی‌ها و...,«سامسونگ دیسپلی» در یک نمایشگاه خصوصی رسانه‌ای...,به گزارش همشهری آنلاین و به نقل از روزنامه هرا...,IT
55332,سه‌شنبه ۲۳ آذر ۱۴۰۰ - ۱۵:۴۸,آیفون تاشو چه زمانی به بازار می‌آید,با موفقیتی که سامسونگ گلکسی زد فولد ۳ و گلکسی ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت تای...,Science
63912,پنجشنبه ۱۶ اردیبهشت ۱۴۰۰ - ۰۲:۱۵,عکس روز | شهردار لندن در رینگ بوکس,صادق خان، شهردار فعلی لندن و نامزد حزب کارگر ب...,صادق خان پاکستانی‌تبار و عضو حزب کارگر انگلیس ...,LifeSkills
36668,سه‌شنبه ۲۳ آذر ۱۴۰۰ - ۱۹:۱۰,ویدئو | آتش سوزی در کارخانه رنگ بارک قم,کارخانه رنگ بارک قم از لحظاتی پیش دچار آتش سوز...,منبع: خبرگزاری صدا و سیما,Society


## Fasttext

In [23]:
class FastText:

    def __init__(self, preprocessor=None, method='skipgram'):
        self.method = method
        self.mean_embed = []
        self.model = None
        self.preprocessor = preprocessor

    def train(self, texts):
        with open('FastText_train.txt', 'w', encoding='utf-8') as file:
            file.write('\n'.join(list(map(lambda doc: ' '.join(doc), texts))))
        self.model = fasttext.train_unsupervised('FastText_train.txt', self.method, minn=2, maxn=5, wordNgrams=10)
        os.remove('FastText_train.txt')
        self.mean_embed = list(map(lambda doc:
                                   np.mean(list(map(lambda word:
                                                    self.model.get_word_vector(word), doc)), axis=0), texts))
        self.mean_embed = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        if self.preprocessor:
            query = self.preprocessor.preprocess(query)
        if type(query) == str:
            query = query.split()
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)
        np.save('FastText_mean_embed.npy', self.mean_embed)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('FastText_mean_embed.npy')

In [24]:
FastText_model = FastText()

In [25]:
FastText_model.train(mini_preprocessed_texts)

In [26]:
# FastText_model.save_FastText_model()
FastText_model.load_FastText_model()

In [ ]:
semantically_similar_words = FastText_model.model.get_nearest_neighbors('آیفون')
print(semantically_similar_words)

[(0.8355935215950012, 'آی\u200cپد'), (0.8261321783065796, 'اپل'), (0.7838040590286255, 'گالکسی'), (0.7690565586090088, 'آیفون\u200cهای'), (0.7669818997383118, 'wwdc'), (0.7611461281776428, 'اسنپ\u200cدراگون'), (0.7572290897369385, 'سامسونگ'), (0.7492721080780029, 'tsmc'), (0.748271644115448, 'هدفون'), (0.7474921941757202, 'bionic')]


In [29]:
queries = []
queries.append("جنگ جهانی")
queries.append(" آیفون ")
queries.append(' اسکوچیچ نگران شد ')
queries.append(' گلکسی سامسونگ ')
queries.append('صعود به جام جهانی')
for query in queries:
    display(FastText_model.predict(query, mini_dataset, k=10))


,date,title,intro,body,category
516,سه‌شنبه ۶ اردیبهشت ۱۴۰۱ - ۰۸:۰۸,خطر واقعی آغاز جنگ جهانی سوم | این خطر را دست ...,وزیر خارجه روسیه درباره خطر واقعی جنگ جهانی سو...,به گزارش همشهری آنلاین، مهر به نقل از گاردین ن...,World
3179,سه‌شنبه ۱۷ اسفند ۱۴۰۰ - ۱۰:۵۷,ببینید | آنجلینا جولی در بین مردم یمن,آنجلینا جولی، بازیگر هالیوود به یمن رفت تا یاد...,به گزارش همشهری آنلاین، مهر به نقل از فاکس‌نیو...,Culture
2037,شنبه ۶ فروردین ۱۴۰۱ - ۰۸:۳۰,مقدماتی جام جهانی| شاگردان کی روش در یک قدمی ...,از قاره آفریقا ۵ تیم راهی جام جهانی قطر خواهد شد.,به گزارش همشهری‌آنلاین در دور رفت مرحله نهایی ...,Sport
601,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۴:۱۱,ادعای وزیر خارجه انگلیس درباره آینده پوتین پس ...,لیز تراس، وزیر امور خارجه انگلیس گفت که حمله ن...,به گزارش همشهری آنلاین، تراس در گفت‌وگو با شبک...,World
2132,جمعه ۱۲ آذر ۱۴۰۰ - ۱۰:۰۴,واکنش بحث‌برانگیز به ۶ هزار کشته ساخت ورزشگاه‌...,رییس کمیته برگزاری مسابقات جام جهانی ۲۰۲۲ قطر ...,به گزارش همشهری‌آنلاین به نقل از اهداف، ناصر ا...,Sport
468,چهارشنبه ۳ فروردین ۱۴۰۱ - ۰۱:۳۰,دبیرکل سازمان ملل خطاب به روسیه: به «جنگ پوچ» ...,آنتونیو گوترش، دبیرکل سازمان ملل از روسیه خواس...,به گزارش همشهری آنلاین به نقل از خبرگزاری فران...,World
564,یکشنبه ۲۹ اسفند ۱۴۰۰ - ۱۷:۵۱,تهدید رئیس‌ جمهوری اوکراین به شروع جنگ جهانی س...,ولودیمیر زلنسکی، رئیس‌جمهوری اوکراین امروز (یک...,به گزارش همشهری آنلاین به نقل از شبکه خبری سی‌...,World
574,چهارشنبه ۲۱ اردیبهشت ۱۴۰۱ - ۰۸:۱۸,جهان آماده جنگ دیگری باشد | لیتوانی خواستار تغ...,گابریلیوس لندسبرگیس وزیر امورخارجه لیتوانی گفت...,به گزارش همشهری آنلاین به نقل از ایرنا، لندسبر...,World
501,یکشنبه ۱ اسفند ۱۴۰۰ - ۰۸:۴۹,هشدار درباره نزدیکی جنگ جهانی سوم | روسیه قصد ...,بوریس جانسون، نخست وزیر انگلیس هشدار داد که وض...,به گزارش همشهری آنلاین به نقل از ایسنا، جانسون...,World
739,شنبه ۶ فروردین ۱۴۰۱ - ۱۲:۳۹,روسیه به مذاکرات صلح جنگ جهانی دوم بازنمی‌گردد...,روسیه چند روز پس از خروج از مذاکرات صلح با ژاپ...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World


,date,title,intro,body,category
3499,دوشنبه ۲۳ اسفند ۱۴۰۰ - ۰۹:۵۳,آیفون ۱۴ با تغییرات اساسی به بازار می‌آید | رو...,سری آینده آیفون یعنی آیفون ۱۴ در چند ماه گذشته...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3451,جمعه ۲۴ دی ۱۴۰۰ - ۱۵:۲۹,آیفون ۱۴ دوربین ۴۸ مگاپیکسلی خواهد داشت,یک گزارش غیررسمی جدید می‌گوید، آیفون ۱۴ پرو به...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3296,دوشنبه ۵ اردیبهشت ۱۴۰۱ - ۱۸:۲۶,فروش آیفون ۱۳ پرو فوق‌العاده بوده است | اپل، ت...,اپل سفارش خود را برای آیفون ۱۳ پرو و آیفون ۱۳ ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3590,یکشنبه ۱۷ بهمن ۱۴۰۰ - ۱۶:۲۴,تصویر | آیفون SE دو ماه دیگر عرضه می‌شود | اول...,اپل کم‌کم آماده می‌شود تا نخستین رویداد سال خو...,به گزارش همشهری آنلاین و به نقل از روزنامه سان...,IT
3439,یکشنبه ۳ بهمن ۱۴۰۰ - ۱۶:۱۲,پیکسل 6A در رویداد ماه مه گوگل معرفی خواهد شد,در سه ماهه چهارم سال ۲۰۲۱، گوگل، پیکسل ۶ و پیک...,به گزارش همشهری آنلاین و به نقل از گیزموچاینا،...,IT
3410,سه‌شنبه ۳ اسفند ۱۴۰۰ - ۱۹:۱۹,اعلام زمان عرضه احتمالی آیفون تاشو | اپل برنام...,بسیاری از تولیدکنندگان پیشرو گوشی‌های هوشمند د...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3508,شنبه ۲۳ بهمن ۱۴۰۰ - ۱۴:۳۲,آیفون ۱۳ پرو مکس به این دلیل از گالکسی اس ۲۲ ا...,سری گالکسی اس ۲۲ سامسونگ یکی از قدرتمندترین گو...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3556,دوشنبه ۴ بهمن ۱۴۰۰ - ۱۷:۲۰,بزرگ‌ترین رویداد تاریخ اپل، پاییز آینده برگزار...,مارک گورمن، تحلیلگر بلومبرگ، اطلاعاتی درباره ب...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3494,دوشنبه ۸ فروردین ۱۴۰۱ - ۱۲:۵۰,جدیدترین برنامه اپل | آی‌پد پرو با تراشه M2 پا...,شرکت اپل به احتمال فراوان پاییز سال جاری از نس...,به گزارش همشهری آنلاین و به نقل از وب‌سایت ورج...,IT
3407,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۲۱:۳۵,دوربین ۲۰۰ مگاپیکسلی برای گالکسی S23 | خیز بلن...,شرکت سامسونگ می‌تواند تلفن همراه مدل گالکسی S2...,به گزارش همشهری آنلاین و به نقل از اندروید پلی...,IT


,date,title,intro,body,category
2217,شنبه ۱۳ شهریور ۱۴۰۰ - ۰۷:۵۳,دوقطبی جاودانه از «کلمبیای کی‌روش» تا «عمان بر...,اگر فکر می‌کنید قرار است دوقطبی کارلوس کی‌روش ...,به گزارش همشهری‌آنلاین، بعد از کی‌روش ۲ سرمربی...,Sport
2317,دوشنبه ۲ اسفند ۱۴۰۰ - ۱۶:۰۷,شوک فرهاد به آبی ها؛ کاپیتان استقلال اخراج شد ...,تمرین امروز استقلال با اتقاق مهمی همراه شد.,به گزارش همشهری آنلاین، در این تمرین، ووریا غف...,Sport
2268,دوشنبه ۹ اسفند ۱۴۰۰ - ۱۱:۲۰,ملی‌پوش فوتبال ایران خطر جنگ را پشت سر گذاشت |...,لژیونر ایرانی تیم زوریا لوهانسک اوکراین از این...,به گزارش همشهری‌آنلاین، در چند روز گذشته با تش...,Sport
577,جمعه ۱۷ اردیبهشت ۱۴۰۰ - ۱۹:۴۱,بحران در هند به خاطر کمبود چوب برای سوزاندن اجساد,شهر بنارس هندوستان که با طغیان اجساد متوفیان ک...,به گزارش همشهری‌آنلاین، شهر بنارس هندوستان که ...,World
598,دوشنبه ۱۲ اردیبهشت ۱۴۰۱ - ۱۴:۴۳,ببینید سیاستمدار سوئدی هتاک به قرآن چگونه پا ب...,سیاستمدار اسلام‌ستیز دانمارکی-سوئدی در حالی که...,به گزارش همشهری آنلاین به نقل از فارس، «راسموس...,World
2379,سه‌شنبه ۲۹ تیر ۱۴۰۰ - ۰۸:۲۰,هفته ۲۸ لیگ برتر | حمله دومنظوره سرخ ها به ماش...,۳هفته تا پایان لیگ باقی مانده و امشب پرسپولیس ...,به گزارش همشهری آنلاین در هفته بیست‌وهشتم از ر...,Sport
1489,چهارشنبه ۴ اسفند ۱۴۰۰ - ۰۷:۳۹,اختلاف خونین ۲ نویسنده و کارگردان | ۲۰ ضربه چا...,اختلاف خونین ۲ کارگردان کار دست‌شان داد.,به گزارش همشهری آنلاین به نقل از روزنامه همشهر...,Society
2052,شنبه ۱۶ مرداد ۱۴۰۰ - ۰۸:۳۴,دایی اعتماد کرد، برانکو و یحیی استفاده!,احمد نوراللهی چطور تبدیل به یکی از ارزشمندترین...,به گزارش همشهری‌آنلاین، در پایان لیگ بیستم و پ...,Sport
2278,چهارشنبه ۱۰ آذر ۱۴۰۰ - ۰۸:۱۴,عکس | درگیری عجیب سرمربی عراق با مهاجمش در محو...,سرمربی عراق به دلیل اقدام عجیب خود در جریان با...,به گزارش همشهری آنلاین در بازی دیشب عراق و عما...,Sport
3977,جمعه ۱۳ اسفند ۱۴۰۰ - ۰۰:۰۵,فرار خانواده ستاره لیگ برتری از اوکراین به لهس...,آندری یارمولنکو، ستاره وستهام، به لهستان سفر خ...,به گزارش همشهری آنلاین به نقل از خبرورزشی، آند...,LifeSkills


,date,title,intro,body,category
3407,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۲۱:۳۵,دوربین ۲۰۰ مگاپیکسلی برای گالکسی S23 | خیز بلن...,شرکت سامسونگ می‌تواند تلفن همراه مدل گالکسی S2...,به گزارش همشهری آنلاین و به نقل از اندروید پلی...,IT
3256,دوشنبه ۱۶ اسفند ۱۴۰۰ - ۲۱:۴۱,مشخصات گالکسی A73 | قدرتمندتر از آنچه که تصورش...,اطلاعات جدید حاکی از آن است که سامسونگ گالکسی ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3581,پنجشنبه ۲۸ بهمن ۱۴۰۰ - ۲۲:۰۵,F23 به زودی عرضه خواهد شد | بازار در انتظار جد...,انتظار می‌رود سامسونگ در هفته‌های آینده مدل‌ها...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3518,سه‌شنبه ۱۷ اسفند ۱۴۰۰ - ۱۷:۱۵,مشخصات گالکسی F23 5G که امروز عرضه شد | یک گوش...,اوایل امروز، سامسونگ اولین دستگاه سری F گالکسی...,به گزارش همشهری آنلاین و به نقل از گیزموچاینا،...,IT
3564,چهارشنبه ۳۱ فروردین ۱۴۰۱ - ۱۹:۱۵,زد فولد۴ با دوربین و تراشه قدرتمند به بازار می...,انتظار می‌رود گالکسی Z Fold۴ سامسونگ بهبودهای ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3461,چهارشنبه ۲۲ دی ۱۴۰۰ - ۱۹:۳۱,گالکسی‌های M۳۳، A۳۳ و A۵۳ در راه هستند,از اواخر سال ۲۰۲۱ در مورد ۳ گوشی میان رده گالک...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3347,یکشنبه ۷ فروردین ۱۴۰۱ - ۱۴:۱۱,تصاویر | گالکسی M33 هفته آینده به بازار می‌آید,گالکسی M33 سامسونگ که اوایل ماه مارس رونمایی ش...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3305,دوشنبه ۴ بهمن ۱۴۰۰ - ۲۰:۲۱,جدیدترین گالکسی سری F آماده ورود به بازار است,به نظر می‌رسد سامسونگ در حال آماده شدن برای عر...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3300,دوشنبه ۲۲ فروردین ۱۴۰۱ - ۱۹:۲۲,وان‌پلاس آس با چیپست دایمنسیتی ۸۱۰۰ آماده ورود...,جدیدترین شایعه منتشرشده نشان می‌دهد که OnePlus...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3582,دوشنبه ۲۰ دی ۱۴۰۰ - ۱۶:۵۷,پایان شایعات در خصوص پرچمدار آینده سامسونگ | ز...,سری گالکسی اس۲۲ سامسونگ مدتی است که بخشی از اف...,به گزارش همشهری آنلاین و به نقل از گجتس۳۶۰، پی...,IT


,date,title,intro,body,category
2037,شنبه ۶ فروردین ۱۴۰۱ - ۰۸:۳۰,مقدماتی جام جهانی| شاگردان کی روش در یک قدمی ...,از قاره آفریقا ۵ تیم راهی جام جهانی قطر خواهد شد.,به گزارش همشهری‌آنلاین در دور رفت مرحله نهایی ...,Sport
2367,یکشنبه ۲۱ فروردین ۱۴۰۱ - ۰۹:۵۵,پیش‌بینی نتایج تیم ملی در جام جهانی قطر| شاگرد...,ساکرنت اقدام به پیش بینی نتایج تیم‌ها در جام ج...,به گزارش همشهری‌آنلاین، در حالی که ما هنوز همه...,Sport
2283,سه‌شنبه ۲۵ آبان ۱۴۰۰ - ۲۱:۵۵,جایگاه ایران در جدول گروه A مقدماتی جام‌ جهانی,ایران با برد ۳ بر صفر مقابل سوریه با ۱۶ امتیا...,,Sport
2166,سه‌شنبه ۲۵ خرداد ۱۴۰۰ - ۲۱:۱۰,صعود تیم برانکو به مرحله نهایی انتخابی جام جها...,تیم ملی فوتبال عمان با تساوی هند مقابل افغانست...,به گزارش همشهری‌آنلاین، تیم ملی هند امروز در گ...,Sport
2132,جمعه ۱۲ آذر ۱۴۰۰ - ۱۰:۰۴,واکنش بحث‌برانگیز به ۶ هزار کشته ساخت ورزشگاه‌...,رییس کمیته برگزاری مسابقات جام جهانی ۲۰۲۲ قطر ...,به گزارش همشهری‌آنلاین به نقل از اهداف، ناصر ا...,Sport
2346,شنبه ۲۰ آذر ۱۴۰۰ - ۱۶:۴۴,ویدیو | جنجال تونسی ها در جام عرب | تخریب صندل...,هواداران تونسی صندلی های ورزشگاه البیت را بعد ...,به گزارش همشهری آنلاین به نقل از فارس، تیم ملی...,Sport
1628,پنجشنبه ۷ بهمن ۱۴۰۰ - ۲۱:۲۲,توئیت فوتبالی شهردار تهران | زاکانی: اول بودن ...,شهردار تهران صعود تاریخی و مقتدارنه تیم ملی فو...,به گزارش همشهری آنلاین، علیرضا زاکانی در توئیت...,City
2067,چهارشنبه ۱۷ آذر ۱۴۰۰ - ۰۹:۴۶,آب و آتش در جام ملت‌های عرب | تقابل جذاب برانک...,تیم‌های تحت هدایت برانکو و کی‌روش تنها یک گام ...,به گزارش همشهری‌آنلاین، رقابت‌های جام ملت‌های ...,Sport
2242,چهارشنبه ۲۸ مهر ۱۴۰۰ - ۱۲:۵۶,فرمت مرحله نهایی مقدماتی جام ملت‌های آسیا ۲۰۲۳...,کنفدراسیون فوتبال آسیا به دلیل شرایط کرونایی ف...,به گزارش همشهری‌آنلاین به نقل از سایت کنفدراسی...,Sport
2389,سه‌شنبه ۶ مهر ۱۴۰۰ - ۱۲:۲۴,عکس | سقوط قهرمان آسیا در آخرین رنکینگ فوتسال ...,تیم ملی کشورمان در آخرین رده‌بندی فوتسال، پس ا...,به گزارش همشهری‌آنلاین، تیم ملی فوتسال کشورمان...,Sport
